### Gaining Actionable Insights from Twitter Data
In this capstone project, we'll implement Structured Streaming to gain insight from streaming Twitter data. The executive team would like to have access to some key business metrics such as:
* The most tweeted hashtag in last 5 minute window
* A map that identifies from where the tweets are coming

First, run the following cell to import the data and make various utilities available for our experimentation.

In [0]:
%run "./Includes/Classroom-Setup"

#### 1.0. Read Streaming Data from Input Source
The input source is a a Kafka feed of Twitter data

For this step you will need to:
0. Use the `format()` operation to specify "kafka" as the type of the stream
0. Specify the location of the Kafka server by setting the option "kafka.bootstrap.servers" with one of the following values (depending on where you are located): 
 * **server1.databricks.training:9092** (US-Oregon)
 * **server2.databricks.training:9092** (Singapore)
0. Indicate which topics to listen to by setting the option "subscribe" to "tweets"
0. Throttle Kafka's processing of the streams
0. Rewind stream to beginning when we restart notebook
0. Load the input data stream in as a DataFrame
0. Select the column `value` - cast it to a `STRING`

In [0]:
from pyspark.sql.functions import col

spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

kafkaServer = "server1.databricks.training:9092"   # US (Oregon)
# kafkaServer = "server2.databricks.training:9092" # Singapore

rawDF = (spark.readStream
  .format("kafka")                                       # Specify "kafka" as the type of the stream
  .option("kafka.bootstrap.servers", kafkaServer)        # Set the location of the kafka server
  .option("subscribe", "tweets")                         # Indicate which topics to listen to
  .option("maxOffsetsPerTrigger", 1000)                  # Throttle Kafka's processing of the streams
  .option("startingOffsets", "earliest")                 # Rewind stream to beginning when we restart notebook
  .load()                                                # Load the input data stream in as a DataFrame
  .select(col("value").cast("STRING"))                   # Select the "value" column and cast to a string
)

#### 2.0. Define a Schema for Parsing the JSON
Simply run the following cell and proceed to the next step.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, ArrayType

twitSchema = StructType([
  StructField("hashTags", ArrayType(StringType(), False), True),
  StructField("text", StringType(), True),   
  StructField("userScreenName", StringType(), True),
  StructField("id", LongType(), True),
  StructField("createdAt", LongType(), True),
  StructField("retweetCount", IntegerType(), True),
  StructField("lang", StringType(), True),
  StructField("favoriteCount", IntegerType(), True),
  StructField("user", StringType(), True),
  StructField("place", StructType([
    StructField("coordinates", StringType(), True), 
    StructField("name", StringType(), True),
    StructField("placeType", StringType(), True),
    StructField("fullName", StringType(), True),
    StructField("countryCode", StringType(), True)]), 
  True)
])

#### 3.0. Create a JSON DataFrame
From the `rawDF` parse out the json subfields using `from_json`. Create a DataFrame that has fields
* `time`
* `json`, a nested field that has all the rest of the data
* promote all `json` subfields to fields.

In [0]:
from pyspark.sql.functions import from_json, expr, col

cleanDF = (rawDF
  .withColumn("json", from_json(col("value"), twitSchema))                      # Add the column "json" by parsing the column "value" with "from_json"
  .select(
    expr("cast(cast(json.createdAt as double)/1000 as timestamp) as time"),     # Cast "createdAt" column properly, call it "time"
    col("json.hashTags").alias("hashTags"),                                     # Promote subfields of "json" column e.g. "json.field" to "field"
    col("json.text").alias("text"),                                             # Repeat for each subfields of "json"
    col("json.userScreenName").alias("userScreenName"),
    col("json.id").alias("id"), 
    col("json.retweetCount").alias("retweetCount"),
    col("json.lang").alias("lang"),
    col("json.favoriteCount").alias("favoriteCount"),
    col("json.user").alias("user"),
    col("json.place.coordinates").alias("coordinates"),
    col("json.place.name").alias("name"),
    col("json.place.placeType").alias("placeType"),
    col("json.place.fullName").alias("fullName"),
    col("json.place.countryCode").alias("countryCode")   
  )
)

#### 4.0. Display Twitter Data as a Table
Click the left-most button in the bottom left corner.

In [0]:
cleanStream = "SS99_clean_p"
display(cleanDF, streamName = cleanStream)

Wait until stream is done initializing...

In [0]:
untilStreamIsReady(cleanStream)

When you are done, stop the stream:

In [0]:
for streamingQuery in spark.streams.active:
  streamingQuery.stop()

#### 5.0. Process Hashtags
In this exercise, we do ETL processing on the `hashTags` column. The goal is to first convert hash tags all to lower case then group tweets and count by hash tags.  You will notice that `hashTags` is an array of hash tags, which you will have to break up (use `explode` function).  The `explode` method allows you to split an array column into multiple rows, copying all the other columns into each new row.

In [0]:
from pyspark.sql.functions import explode, lower

twitCountsDF = (cleanDF                          # Start with "cleanDF"
  .withColumn("hashTag", explode("hashTags"))    # Explode the array "hashTags" into "hashTag"
  .withColumn("hashTag", lower(col("hashTag")))  # Convert "hashTag" to lower case
  .groupBy("hashTag")                            # Aggregate by "hashTag"
  .count()                                       # For the aggregate, produce a count  
  .orderBy(col("count").desc())                  # Sort by "count"
  .limit(25)                                     # Limit the result to 25 records
)

#### 6.0. Plot Counts of the Top 25 Most Popular Hashtags
Under **Plot Options**, use the following:
* **Keys:** `hashTag`
* **Values:** `count`

In **Display type**, use **Pie Chart** and click **Apply**.  Once you apply the plot options, be prepared to increase the size of the plot graphic using the resize widget in the lower right corner of the graphic area.

In [0]:
twitStream = "SS99_twit_p"
display(twitCountsDF, streamName = twitStream)

Wait until stream is done initializing...

In [0]:
untilStreamIsReady(twitStream)

When you are done, stop the stream:

In [0]:
for streamingQuery in spark.streams.active:
  streamingQuery.stop()

#### 7.0. Read in File with Two-to-Three Letter Country Codes</h3>
For this next part we are going to take a look at the number of requests per country. To get started, we first need a lookup table that will give us the 3-character country code.
1. Read in the file at `/mnt/training/countries/ISOCountryCodes/ISOCountryLookup.parquet`
2. We will be interested in the `alpha2Code` and `alpha3Code` fields later

In [0]:
countryCodeDF = spark.read.parquet("/mnt/training/countries/ISOCountryCodes/ISOCountryLookup.parquet")

#### 8.0. Join Tables &amp; Aggregate By Country
In `cleanDF`, there is a `countryCode` field. However, it is in the form of a two-letter country code.  The `display` map expects a three-letter country code.  In order to retrieve tweets with three-letter country codes, we will have to join `cleanDF` with `countryCodesDF`.

In [0]:
mappedDF = (cleanDF
  .filter(col("countryCode").isNotNull())                                      # Filter out any nulls for "countryCode"
  .join(countryCodeDF, cleanDF["countryCode"] == countryCodeDF["alpha2Code"])  # Join the two tables on "countryCode" and "alpha2Code"
  .groupBy(col("alpha3Code"))                                                  # Aggregate by country, "alpha3Code"
  .count()                                                                     # Produce a count of each aggregate
)

#### 9.0. Plot the Tweet Counts on a World Map

Under **Plot Options**, use the following:
* **Keys:** `alpha3Code`
* **Values:** `count`

In **Display type**, use **World map** and click **Apply**.

<img src="https://files.training.databricks.com/images/eLearning/Structured-Streaming/plot-options-map-06.png"/>

In [0]:
mappedStream = "SS99_mapped_p"
display(mappedDF, streamName = mappedStream)

Wait until stream is done initializing...

In [0]:
untilStreamIsReady(mappedStream)

#### 10.0. Write the stream to an in-memory table
0. Use appropriate `format`
0. For this exercise, we want to append new records to the results table
0. Gives the query a name
0. Start the query
0. Assign the query to `mappedTablePython`

In [0]:
mappedQuery = (mappedDF 
 .writeStream                           # From the DataFrame get the DataStreamWriter
 .format("memory")                      # Specify the sink format as "memory"
 .outputMode("complete")                # Configure the output mode as "complete"
 .queryName("mappedTablePython")        # Name the query "mappedTablePython"
 .start()                               # Start the query
)

Wait until stream is done initializing...

In [0]:
untilStreamIsReady("mappedTablePython")

Make sure to stop the stream before continuing.

In [0]:
stopAllStreams()

#### 11.0. Use SQL Syntax to Display a Few Rows</h2>
Author a basic SQL query to display all columns, limiting the response to 10 rows.
We supplied `.queryName()` in the `writeStream` query above so that we would be able to treat `<queryName>` as a table for the sake of authoring SQL queries!

In [0]:
%sql
SELECT * FROM mappedTablePython LIMIT 10;

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [0]:
%run ./Includes/Classroom-Cleanup